In [1]:
## note: "last 20yr" duration adjustments only use data from 1996-2015 (so that results are stable with updates)
from __future__ import division
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta  ## use timedelta to increment targ by 1 year!!!!
#from datetime import timedelta  ## use this to increment targ by 1 year!!!!
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices
#import random

import sys
import os
os.chdir('C:/BankRisk')

In [2]:
dur20y = pd.DataFrame(columns=['dur','adj'])  ## used at end to save last 20y act vs naive durations of ladders
dur20y

Empty DataFrame
Columns: [dur, adj]
Index: []

In [3]:
sp30y = pd.read_csv("ccs_all365_longhist_30yr_201812.csv",index_col=0)
sp30y.index = pd.to_datetime(sp30y.index)
sp30y.columns = sp30y.columns.astype(int)

In [4]:
sp30y.tail(2)

0         1         2         3         4         5    \
Date                                                                     
2019-02-01  2.433333  2.529111  2.636830  2.713854  2.743791  2.765180   
2019-03-01  2.443472  2.524386  2.591781  2.641865  2.674909  2.701243   

                 6         7         8         9    ...       351       352  \
Date                                                ...                       
2019-02-01  2.773349  2.768301  2.755371  2.737879  ...  2.831297  2.831703   
2019-03-01  2.711992  2.707242  2.694919  2.677915  ...  2.753792  2.754335   

                 353       354      355       356       357       358  \
Date                                                                    
2019-02-01  2.832109  2.832515  2.83292  2.833326  2.833731  2.834137   
2019-03-01  2.754876  2.755418  2.75596  2.756501  2.757043  2.757584   

                 359       360  
Date                            
2019-02-01  2.834542  2.834947  
2019-03-01  2.758125  2.758667  

[2 rows x 361 columns]

In [5]:
#df_sp = pd.read_csv("df_spot_longhist_30y.csv",index_col=0)
df_sp = pd.read_csv("df_spot_fwd3_longhist_30y_201812.csv",index_col=0)
df_sp['Date'] = pd.to_datetime(df_sp['Date'])
df_sp.reset_index(drop=False,inplace=True)
df_sp.set_index(['Desc','Date'],drop=True,inplace=True)
df_sp.columns = df_sp.columns.astype(int)
df_sp.tail(2)

0         1         2         3         4         5    \
Desc Date                                                                     
fwd3 2019-02-01  2.704689  2.799610  2.828251  2.803902  2.766010  2.698700   
     2019-03-01  2.633179  2.710161  2.752791  2.754361  2.716798  2.645732   

                      6         7         8         9    ...       351  \
Desc Date                                                ...             
fwd3 2019-02-01  2.621852  2.554882  2.507392  2.560516  ...  3.044601   
     2019-03-01  2.566595  2.499430  2.447220  2.486227  ...  3.040314   

                      352       353       354       355       356       357  \
Desc Date                                                                     
fwd3 2019-02-01  3.039878  3.034423  3.029628  3.036978  3.045091  3.052570   
     2019-03-01  3.036102  3.031590  3.027790  3.035515  3.043576  3.050959   

                 358  359  360  
Desc Date                       
fwd3 2019-02-01  NaN  NaN  NaN  
     2019-03-01  NaN  NaN  NaN  

[2 rows x 361 columns]

In [6]:
############################################################
############################################################
## ladder dur, MV section:

for x in [1,2,3,4,5,6,7,8,9,10,12,15,20,25,30]:
#if 1==1: ## used for testing new code... runs 5yr only
#  x = 5

  mt = 12*x  ## swap maturity: use 1,2,3,4,5,6,7,8,9,10,12,15,20 (all * 12 to convert to months)

  ci = [x for x in np.arange(1,mt+1) if x%6==(mt%6)]  # get coupon dates for new swap
  #ci[-5:]
  t2 = pd.Series(np.zeros(mt),index=np.arange(1,mt+1))  # temp table for processing
  #t2[-5:]
  ## start sections to calc ladder durations and MVs:
  #ci[-5:]
  for i in np.arange(mt): # fill up cash flows in 1st possible ladder
    cp = sp30y.ix[i,mt]/2 # get coupon
    for j in np.arange(1,mt): # move prior month cash flows up by 1 month
      t2.loc[j] = t2.loc[j+1]
    t2.loc[mt] = 0        # zero out last cash flow
    t2[ci] = t2[ci]+cp    # assign coupons to payment periods
    t2[mt] = t2[mt]+100   # add return of principal to maturity period
  # tm table for holding cfs:
  tm = pd.DataFrame(t2,columns=[sp30y.index[mt-1].strftime('%Y-%m-%d')]).T
  tm.index = pd.to_datetime(tm.index)
  #tm
  for i in np.arange(mt,len(sp30y)): # process another month:
    for j in np.arange(1,mt): # move prior month cash flows up by 1 month
      t2.loc[j] = t2.loc[j+1]
    t2.loc[mt] = 0        # zero out last cash flow
    cp = sp30y.ix[i,mt]/2 # get coupon for new swap
    t2[ci] = t2[ci]+cp    # add new coupons to payment periods
    t2[mt] = t2[mt]+100   # add return of principal to maturity period
    d2 = pd.DataFrame(t2,columns=[sp30y.index[i].strftime('%Y-%m-%d')]).T
    d2.index = pd.to_datetime(d2.index)
    tm=tm.append(d2)
  tm.to_csv(str(mt)+'mLadder_cfs_longhist2.csv')
  #str(mt)+'mLadder_cfs_longhist2.csv'

  # calc MacDurs and MVs:
  ts = list(tm.columns)
  ts = [x/12 for x in ts]
  #ts[-5:]  # list of yrs for dur calcs
  dv = pd.DataFrame(tm[[1,2,3,4]])
  dv.columns = ['mv','mvpc','dur','cpn']
  dv[['mv','mvpc','dur','cpn']] = [0,0,0,0]
  #dv.head()
  ## calc dur, mv, mv% for all ladders:
  for i in np.arange(len(tm)):
    pv= [x*y for x,y in zip(tm.ix[i][0:mt],list(df_sp.loc['df',tm.index[i]])[1:mt+1])] ## mult 2 vectors: cf*df
    pvt= [x*y for x,y in zip(pv,ts)]
    dv.ix[i]['dur'] = sum(pvt)/sum(pv)
    dv.ix[i]['mv'] = sum(pv) - 100*mt
    dv.ix[i]['mvpc'] = sum(pv) / (100*mt)
    dv.ix[i]['cpn'] = sp30y.ix[dv.index[i]][mt] # pull coupon for swap maturity mt
  ## add fed funds column to first section for used in case study later:
  if (mt == 12)&('ff' not in list(dv.columns)):
    dvcol = ['mv','mvpc','dur','cpn']
    dv = dv.merge(sp30y[[0]],how='left',left_index=True,right_index=True)
    dv.rename(columns={0:'ff'},inplace=True)
    dv = dv[['ff']+dvcol]
  
  #dv.tail(3)
  ## calc MV3m:  
  b3 = mt/12  # balance per month repricing
  mv3 = pd.DataFrame(sp30y[3])
  mv3.rename(columns={3:'rt'},inplace=True)
  df3 = mv3.merge(df_sp.loc['df'][[1,2,3]],how='left',left_index=True,right_index=True)
  df3['cf1'] = df3['rt'].shift(2) * b3 + mt*100/3
  df3['cf2'] = df3['rt'].shift(1) * b3 + mt*100/3
  df3['cf3'] = df3['rt'] * b3 + mt*100/3
  df3['mv3'] = df3[1]*df3['cf1']+df3[2]*df3['cf2']+df3[3]*df3['cf3'] - mt*100
  #df3.head()
  dv['mv3'] = df3['mv3']
  #dv.head()
  roll3m = pd.DataFrame(sp30y[3].rolling(3, min_periods=3).mean())
  roll3m.columns = ['roll3m']
  sp30ya = sp30y[mt].rolling(mt, min_periods=mt).mean()
  sp30ya.dropna(inplace=True)
  #sp30ya.head()
  #roll3m = pd.DataFrame(pd.rolling_mean(sp30y[3],3, min_periods=3))
  #roll3m.columns = ['roll3m']
  #roll3m.head()
  dv['roll'+str(mt)+'m'] = sp30ya
  dv = dv.merge(roll3m, how='left',left_index=True, right_index=True)
  #dv.tail()
  dv.to_csv('durmv_'+str(mt)+'mladder_longhist2.csv')
  #'durmv_'+str(mt)+'mladder_longhist2.csv'
  ### save last calc 20y dur vs naive dur in a table
  ###
  ###########################################################################################################
  ### NOTE: Change this code to use 20yrs up to Dec-2015 only!!!!!!!!!!! (so that book results are reproduced)
  ###########################################################################################################
  ###
  #dur20y.loc[mt] = [ dv['dur'][-240:].mean(),  dv['dur'][-240:].mean()/(mt/12/2) ]
  dur20y.loc[mt] = [ dv.loc['1996':'2015','dur'].mean(),  dv.loc['1996':'2015','dur'].mean()/(mt/12/2) ]
  dur20y.sort_index(inplace=True)
  #dur20y
  ## save copies of results according to orig maturity:
  if mt == 12:
    dv12 = dv.copy()
  elif mt == 24:
    dv24 = dv.copy()
  elif mt == 36:
    dv36 = dv.copy()
  elif mt == 48:
    dv48 = dv.copy()
  elif mt == 60:
    dv60 = dv.copy()
  elif mt == 72:
    dv72 = dv.copy()
  elif mt == 84:
    dv84 = dv.copy()
  elif mt == 96:
    dv96 = dv.copy()
  elif mt == 108:
    dv108 = dv.copy()
  elif mt == 120:
    dv120 = dv.copy()
  elif mt == 144:
    dv144 = dv.copy()
  elif mt == 180:
    dv180 = dv.copy()
  elif mt == 240:
    dv240 = dv.copy()
  elif mt == 300:
    dv300 = dv.copy()
  elif mt == 360:
    dv360 = dv.copy()


In [7]:
### STOP HERE!!! run all maturities before proceeding

In [8]:
dur20y.to_csv('durladder_last20y_adjmnts.csv')

In [9]:
#voldet.columns = pd.MultiIndex.from_product([[prn], list(voldet.columns)])
dv12.columns = pd.MultiIndex.from_product([['dv12'],list(dv12.columns)])
dv24.columns = pd.MultiIndex.from_product([['dv24'],list(dv24.columns)])
dv36.columns = pd.MultiIndex.from_product([['dv36'],list(dv36.columns)])
dv48.columns = pd.MultiIndex.from_product([['dv48'],list(dv48.columns)])
dv60.columns = pd.MultiIndex.from_product([['dv60'],list(dv60.columns)])
dv72.columns = pd.MultiIndex.from_product([['dv72'],list(dv72.columns)])
dv84.columns = pd.MultiIndex.from_product([['dv84'],list(dv84.columns)])
dv96.columns = pd.MultiIndex.from_product([['dv96'],list(dv96.columns)])
dv108.columns = pd.MultiIndex.from_product([['dv108'],list(dv108.columns)])
dv120.columns = pd.MultiIndex.from_product([['dv120'],list(dv120.columns)])
dv144.columns = pd.MultiIndex.from_product([['dv144'],list(dv144.columns)])
dv180.columns = pd.MultiIndex.from_product([['dv180'],list(dv180.columns)])
dv240.columns = pd.MultiIndex.from_product([['dv240'],list(dv240.columns)])
dv300.columns = pd.MultiIndex.from_product([['dv300'],list(dv300.columns)])
dv360.columns = pd.MultiIndex.from_product([['dv360'],list(dv360.columns)])

In [10]:
dv12.head()

dv12                                                     \
                  ff         mv      mvpc       dur       cpn       mv3   
1965-12-01  4.380000   9.872790  1.008227  0.534467  5.493205  4.784516   
1966-01-01  4.481389   9.288667  1.007741  0.534508  5.463205  4.955346   
1966-02-01  4.663889   9.148350  1.007624  0.534340  5.713205  5.366464   
1966-03-01  4.714583  10.391184  1.008659  0.534491  5.503205  5.672123   
1966-04-01  4.734861  11.174960  1.009312  0.534420  5.533205  5.637323   

                                
             roll12m    roll3m  
1965-12-01  4.854038  4.998342  
1966-01-01  4.916538  5.187601  
1966-02-01  4.990705  5.376861  
1966-03-01  5.053205  5.447833  
1966-04-01  5.117372  5.457972

In [11]:
dvtot = dv12.copy()
dvtot = dvtot.merge(dv24,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv36,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv48,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv60,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv72,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv84,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv96,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv108,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv120,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv144,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv180,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv240,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv300,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv360,how='left',left_index=True,right_index=True)

In [12]:
dvtot.tail()

dv12                                                    \
                  ff        mv      mvpc       dur       cpn       mv3   
2018-11-01  2.230556  4.486110  1.003738  0.538012  2.915200  2.400847   
2018-12-01  2.301528  5.126021  1.004272  0.537990  2.844684  2.556489   
2019-01-01  2.433333  5.893864  1.004912  0.537946  2.763750  2.855265   
2019-02-01  2.433333  6.740588  1.005617  0.537896  2.691231  3.028021   
2019-03-01  2.443472  7.203299  1.006003  0.537861  2.627749  2.948867   

                                     dv24            ...      dv300            \
             roll12m    roll3m         mv      mvpc  ...        mv3  roll300m   
2018-11-01  2.490453  2.520946  -2.141593  0.999108  ...  60.021184  4.868117   
2018-12-01  2.572610  2.668600   2.342340  1.000976  ...  63.912216  4.856355   
2019-01-01  2.635018  2.774569   6.154603  1.002564  ...  71.381622  4.844293   
2019-02-01  2.675559  2.783861   9.065558  1.003777  ...  75.700527  4.831162   
2019-03-01  2.696935  2.722809  11.731582  1.004888  ...  73.721677  4.816091   

                            dv360                                            \
              roll3m           mv      mvpc        dur       cpn        mv3   
2018-11-01  2.520946  5787.400678  1.160761  10.353694  3.243900  72.025421   
2018-12-01  2.668600  6954.664988  1.193185  10.533410  2.967632  76.694660   
2019-01-01  2.774569  7492.776349  1.208133  10.608104  2.852143  85.657947   
2019-02-01  2.783861  7569.306321  1.210259  10.621379  2.834947  90.840633   
2019-03-01  2.722809  7955.632713  1.220990  10.673695  2.758667  88.466013   

                                
            roll360m    roll3m  
2018-11-01  5.532963  2.520946  
2018-12-01  5.514512  2.668600  
2019-01-01  5.496184  2.774569  
2019-02-01  5.476781  2.783861  
2019-03-01  5.457194  2.722809  

[5 rows x 106 columns]

In [13]:
dvtot.to_csv('ladder_durmv_allmat_longhist_201812.csv')
len(dvtot) # 640

640

In [14]:
## methods to access multiindex data:
dvtot['dv24','mv'][10:15]  # one column

1966-10-01          NaN
1966-11-01          NaN
1966-12-01    27.638363
1967-01-01    35.049880
1967-02-01    34.315623
Name: (dv24, mv), dtype: float64

In [15]:
## methods to access multiindex cell:
dvtot.ix['2015-09-01']['dv24','dur']  # one cell  (loc works too)

1.0371136056624677

In [16]:
dur20y

dur       adj
12   0.537237  1.074474
24   1.020231  1.020231
36   1.488679  0.992452
48   1.940960  0.970480
60   2.376087  0.950435
72   2.793793  0.931264
84   3.193705  0.912487
96   3.575723  0.893931
108  3.939583  0.875463
120  4.288722  0.857744
144  4.949200  0.824867
180  5.838247  0.778433
240  7.075556  0.707556
300  8.089359  0.647149
360  8.956141  0.597076